<a href="https://colab.research.google.com/github/Ramprabu95/AI_based_image_compression/blob/main/Learning_convolutional_Networks_for_Content_weighted_Image_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Convolutional Networks for Content Weighted Image Compression

# **Introduction:**


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
def conv(ni, nf, kernal_size=3, stride=1, padding=1, **kwargs):
    _conv = nn.Conv2d(ni, nf, kernel_size=kernal_size,stride=stride,padding=padding, **kwargs)
    return _conv

class ResBlock(nn.Module):
    def __init__(self, ni, nh=128):
        super().__init__()
        self.conv1 = conv(ni, nh)
        self.conv2 = conv(nh, ni)
    def forward(self, x):
        return x  + self.conv2(F.relu(self.conv1(x)))

class Encoder(nn.Module):
    def __init__(self,return_imp_map=False):
        super(Encoder, self).__init__()
        self.return_imp_map = return_imp_map
        self.stem = nn.Sequential(conv(3, 128, 8, 4, 2), relu,
                                   ResBlock(128), relu,
                                   conv(128, 256, 4, 2, 1), relu,
                                   ResBlock(256), relu,
                                   ResBlock(256), relu)
    def forward(self,x):
        stem = self.stem(x)
        return(stem)